Only run the following cell once

In [1]:
# Only run this cell once
import os
moduledir = os.path.dirname( os.getcwd() )

os.chdir(moduledir) # Move up one directory so we can import modules

# BLG in STM

To generate a $dI/dV$ spectrum over all gate voltages, we first import the `TunnelingExperiments` module. A tunneling experiment performed on BLG is provided by the object `TunnelingExperiments.BLGinSTM`. The parameters it takes are as follows

* The distance between the top gate (tip) and sample `d1` in units nm
* The distance between the bottom gate and sample `d2` in units nm
* The *relative* permittivity between the tip and sample `e1`
* The *relative* permittivity between the bottom gate and sample `e2`
* Temperature `T` in Kelvin
* The work function of the tip `Wtip` in units eV

A common example is to have a layer BLG on a ~20 nm thick piece of hBN on a 285 nm thick SiO$_2$ oxide. This sets `d_2=20+285=305` nm. The relative permittivity of both hBN and SiO$_2$ is roughly 4, so we set `e2=4`. The tip height varies substantially, but in this example, we'll set `d_1=1` nm. Since the tip sits in vacuum, `e1=1`. If the tip is made of Pt-Ir, then `Wtip=5` eV approximately.

In [2]:
import numpy as np

from TunnelingExperiment import TunnelingExperiments

In [ ]:
d1_array = [0.1,0.2,0.5,1,2,5,10] # nm between tip and sample
d2 = 305 # nm between backgate and sample
e1 = 1 # relative permittivity between tip and sample
e2 = 4 # relative permittivity between backgate and sample
T = 0 # Kelvin
Wtip = 5 # eV for work function of the tip


for d1 in d1_array:
    experiment = TunnelingExperiments.BLGinSTM(d1,d2,e1,e2,T,Wtip)

    VTrange = [-0.2,0.2]
    num_vts_100 = 3
    VT = np.linspace(VTrange[0],VTrange[-1],num=num_vts_100*100)

    VBrange = [-45,45]
    num_vbs_100 = 3
    VB = np.linspace(VBrange[0],VBrange[-1],num=num_vbs_100*100)

    experiment.generate_tunnelcurrent(VTrange,num_vts_100,VBrange,num_vbs_100,method='DasSarma')
    experiment.plot_dIdV(show=False,save=True)

Computing tunnel currents


C:\python\lib\site-packages\scipy\integrate\quadpack.py:385: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  warnings.warn(msg, IntegrationWarning)


C:\python\lib\site-packages\scipy\integrate\quadpack.py:385: IntegrationWarning: Extremely bad integrand behavior occurs at some points of the
  integration interval.
  warnings.warn(msg, IntegrationWarning)


C:\python\lib\site-packages\scipy\integrate\quadpack.py:385: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  warnings.warn(msg, IntegrationWarning)


C:\Users\holdm\Documents\UW-Madison\BrarGroup\Master\BLGinSTM\Materials\Graphene.py:161: RuntimeWarning: divide by zero encountered in true_divide
  propdkp2 = 2 + (self.g1**2 + u**2)/np.sqrt(disc)
C:\Users\holdm\Documents\UW-Madison\BrarGroup\Master\BLGinSTM\Materials\Graphene.py:162: RuntimeWarning: divide by zero encountered in true_divide
  propdkm2 = 2 - (self.g1**2 + u**2)/np.sqrt(disc)


C:\Users\holdm\Documents\UW-Madison\BrarGroup\Master\BLGinSTM\TunnelingExperiment\TunnelingExperiments.py:282: RuntimeWarning: divide by zero encountered in true_divide
  dIdV_plot = plt.imshow(dIdV/IV,cmap=cm.RdYlGn,origin='lower',
C:\Users\holdm\Documents\UW-Madison\BrarGroup\Master\BLGinSTM\TunnelingExperiment\TunnelingExperiments.py:282: RuntimeWarning: invalid value encountered in true_divide
  dIdV_plot = plt.imshow(dIdV/IV,cmap=cm.RdYlGn,origin='lower',


Computing tunnel currents
Computing tunnel currentshedd
Computing tunnel currentshedd


# Total Current

The total current, as given in Chen 2008, is

$$I =\frac{4\pi eA_c}{\hbar}|M(0)|^2\rho_T \int_{-\infty}^{\infty}[f(\epsilon-eV_T)-f(\epsilon)]\rho_{BLG}(\epsilon_F+\epsilon)\exp{(\kappa_0(\epsilon - \frac{1}{2}eV_T) s/2\bar\phi)}\quad d\epsilon $$

where

* $\bar\phi = W_T - \frac{e}{2}V_T$ is the average work function of the tip and graphene
* $\kappa_0=\frac{\sqrt{2m\bar\phi}}{\hbar}$ is a decay constant
* $s$ is the distance between the tip and sample
* $f(\epsilon)=[1+\exp{\epsilon/k_BT}]^{-1}$ is the Fermi-Dirac distribution